In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [7]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [8]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset['Purchased']

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)


In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 



Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [15]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'random'}: 0.8299236386470121


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[71 14]
 [ 9 40]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.89      0.84      0.86        85
           1       0.74      0.82      0.78        49

    accuracy                           0.83       134
   macro avg       0.81      0.83      0.82       134
weighted avg       0.83      0.83      0.83       134



In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


np.float64(0.8258103241296518)

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002957,0.000114,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.003176,0.000738,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.009316,0.004280,0.012501,0.001658,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.813511,0.795256,0.832483,0.847020,0.822907,0.017534,7
3,0.006299,0.000536,0.011864,0.001351,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.867478,0.804628,0.756031,0.793565,0.905069,0.825354,0.053619,5
4,0.005874,0.000782,0.017755,0.006833,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.849794,0.870047,0.851527,0.813179,0.841025,0.845115,0.018554,2
5,0.015292,0.016666,0.033726,0.018814,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.808927,0.852030,0.718497,0.832483,0.885265,0.819440,0.056329,8
6,0.013298,0.016778,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.002356,0.000673,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.008136,0.001259,0.010232,0.002339,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.868752,0.814409,0.851527,0.865054,0.840901,0.026381,4
9,0.004998,0.000924,0.008047,0.001595,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.864871,0.831253,0.759244,0.813179,0.849057,0.823521,0.036497,6


In [25]:
#save model
import pickle
filename = "finalized_model_classification.sav"
pickle.dump(grid,open(filename,"wb"))
load_model = pickle.load(open(filename,"rb"))
age_input=float(input("Age:"))
salary_input=float(input("BMI:"))
sex_male_input=int(input("Sex Male 0 or 1:"))


Age: 56
BMI: 23
Sex Male 0 or 1: 1


In [27]:
result = load_model.predict([[age_input, salary_input, sex_male_input]])
print("Future_Prediction={}".format(result))

Future_Prediction=[1]


Future_Prediction=[1]
